In [2]:
import heapq #El módulo heapq implementa colas de prioridad (heaps)

In [3]:
class Node:
    def __init__(self, position, parent=None, actions=None, path_cost=0): 
        self.position = position
        self.parent = parent
        self.actions = actions if actions is not None else []
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

class Problem:
    #DEFINA la Class problem como lo considere necesario, puede basarse del ejemplo de Bucharest
    def __init__(self, maze, start, end):
        self.maze = maze
        self.start = start
        self.end = end
        # Diccionario que mapea los movimientos (fila, columna) a las acciones correspondientes
        self.actions = {
            (-1, 0): "Up",      # Mover hacia arriba (fila-1)
            (1, 0): "Down",     # Mover hacia abajo (fila+1)
            (0, -1): "Left",    # Mover a la izquierda (col-1)
            (0, 1): "Right"     # Mover a la derecha (col+1)
        }

In [8]:
def find_exit(maze):
    start = (1, 1)  # Posición inicial basado en la documentación suministrada
    end = (1, 6)    # Posición de la salida basado en la documentación suministrada

    #DEFINA el conjunto de actions posibles

    problem=Problem(maze ,start, end)#COMPLETE LA DEFINICIÓN DEL OBJETO Y ADAPTELO EN LOS PUNTOS QUE LO REQUIERAN

    def manhatan_distance(pos, end):
        return abs(pos[0] - end[0]) + abs(pos[1] - end[1])  # Distancia de Manhattan

    def get_neighbors(pos):  #ESTA ES LA FUNCIÓN QUE DEBERIA AJUSTAR PARA HACER TRACKING DE LOS MOVIMIENTOS (Up, Down, Right, Left)
        neighbors = [] #lista de vecinos con sus acciones
        for move, action_name in problem.actions.items():  #Iteramos sobre movimientos y nombres de acciones
            neighbor = (pos[0] + move[0], pos[1] + move[1])
            if maze[neighbor[0]][neighbor[1]] != "#": #si el vecino es diferente a "#" pared agregarlo a la lista
                neighbors.append((neighbor, action_name))  #Guardamos tupla (posición, acción)
        return neighbors

    start_node = Node(start, path_cost=0)
    frontier = [(manhatan_distance(start, end), start_node)]
    heapq.heapify(frontier) #Convierte la lista frontier en una cola de prioridad (heap)
    reached = {start: start_node}

    while frontier:
        _, node = heapq.heappop(frontier)
        if node.position == end:
            return reconstruct_path(node)

        for neighbor, action in get_neighbors(node.position):  #Ahora recibimos posición y acción
            new_cost = node.path_cost + 1
            if neighbor not in reached or new_cost < reached[neighbor].path_cost:
                new_actions = node.actions + [action]  #Agregamos la nueva acción al historial
                reached[neighbor] = Node(neighbor, parent=node, actions=new_actions, path_cost=new_cost)
                heapq.heappush(frontier, (manhatan_distance(neighbor, end), reached[neighbor]))

    return None  # No se encontró salida

In [7]:
def reconstruct_path(node):  #AJUSTAR FUNCIONES PARA ADEMAS DE LAS POSICIONES, MOSTRAR LAS ACCIONES TOMADAS
    path = []
    actions = node.actions  #El nodo final ya tiene todo el historial de acciones acumuladas
    
    while node:
        path.append(node.position)
        node = node.parent

    path.reverse()
    return path, actions  #Retornamos tanto el camino como las acciones

In [6]:
maze = [
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "S", "#", " ", "#", " ", "E","#"],
    ["#", " ", " ", " ", "#", " ", " ","#"],
    ["#", " ", "#", " ", " ", " ", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"]
]
path, actions = find_exit(maze)
print("Path to exit:", path)
print("Actions taken:", actions)

Path to exit: [(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6)]
Actions taken: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']


**UNA VEZ SOLUCIONADO EL EJERCICIO, RESPONDA:**

1. ¿Cómo cambia el comportamiento del algoritmo si cambiamos la función de costo?
2. ¿Qué sucede si hay múltiples salidas en el laberinto? ¿Cómo podrías modificar el algoritmo para manejar esto?
3. Modifica el laberinto por uno mas grande y con otros tipos de obstaculos a demas de pared ¿que limitación encuentras en el algoritmo? Nota: Resuelve este problema en una celda aparte, con el fin de mantener la integridad de tu código original

### Respuesta 1: ¿Cómo cambia el comportamiento del algoritmo si cambiamos la función de costo?

El algoritmo actual usa **costo uniforme** de 1 (`new_cost = node.path_cost + 1`). Si cambiamos la función de costo:

- **Costos variables**: El algoritmo preferirá rutas más "baratas", aunque sean más largas en pasos.
- **Costos por tipo de terreno**: Buscará la ruta con menor costo total, no la más corta en distancia.
- **Mantiene optimalidad**: A* sigue siendo óptimo si la heurística es admisible y consistente.

### Respuesta 2: ¿Qué sucede si hay múltiples salidas? ¿Cómo modificar el algoritmo?

**Comportamiento actual:** El algoritmo encuentra una única salida fija `end = (1, 6)`. Con múltiples salidas, encontraría solo la primera según su orden de exploración.

**Modificaciones:**

1. **Encontrar salida más cercana**: Cambiar condición de parada a verificar si `maze[node.position[0]][node.position[1]] == "E"`.

2. **Encontrar la óptima**: Escanear todas las salidas "E" y usar heurística a la salida más cercana:
   ```python
   def manhattan_to_nearest_exit(pos, exits):
       return min(abs(pos[0] - exit[0]) + abs(pos[1] - exit[1]) for exit in exits)
   ```

### Respuesta 3: Laberinto más grande con diferentes tipos de obstáculos

**Limitación encontrada:**

El algoritmo solo distingue entre **pared ("#")** y **espacio libre** (costo 1 uniforme). No puede manejar terrenos con diferentes costos (agua, arena, lodo, etc.).

**Problema:** `new_cost = node.path_cost + 1` siempre suma 1, sin importar el tipo de terreno.

A continuación, ejemplo con terrenos variables y la solución modificada:

In [ ]:
# Ejemplo de laberinto más grande con diferentes tipos de obstáculos
# Leyenda:
# "#" = Pared (intransitable)
# "S" = Inicio
# "E" = Salida
# "." = Camino normal (costo 1)
# "W" = Agua (costo 5)
# "A" = Arena (costo 3)
# "M" = Lodo (costo 7)

def find_exit_with_costs(maze):
    start = None
    end = None
    
    # Buscar inicio y fin automáticamente
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == "S":
                start = (i, j)
            elif maze[i][j] == "E":
                end = (i, j)
    
    # Diccionario de costos según el tipo de terreno
    terrain_costs = {
        ".": 1,   # Camino normal
        "W": 5,   # Agua
        "A": 3,   # Arena
        "M": 7,   # Lodo
        "S": 1,   # Inicio
        "E": 1    # Salida
    }
    
    actions = {
        (-1, 0): "Up",
        (1, 0): "Down",
        (0, -1): "Left",
        (0, 1): "Right"
    }
    
    def manhatan_distance(pos, end):
        return abs(pos[0] - end[0]) + abs(pos[1] - end[1])
    
    def get_neighbors(pos):
        neighbors = []
        for move, action_name in actions.items():
            neighbor = (pos[0] + move[0], pos[1] + move[1])
            # Verificar límites y que no sea pared
            if (0 <= neighbor[0] < len(maze) and 
                0 <= neighbor[1] < len(maze[0]) and 
                maze[neighbor[0]][neighbor[1]] != "#"):
                
                # Obtener el costo del terreno (AQUÍ ESTÁ LA MODIFICACIÓN CLAVE)
                terrain = maze[neighbor[0]][neighbor[1]]
                cost = terrain_costs.get(terrain, 1)  # Costo por defecto 1
                neighbors.append((neighbor, action_name, cost))
        return neighbors
    
    start_node = Node(start, path_cost=0)
    frontier = [(manhatan_distance(start, end), start_node)]
    heapq.heapify(frontier)
    reached = {start: start_node}
    
    while frontier:
        _, node = heapq.heappop(frontier)
        if node.position == end:
            return reconstruct_path(node)
        
        for neighbor, action, cost in get_neighbors(node.position):  # Ahora incluye el costo
            new_cost = node.path_cost + cost  # MODIFICACIÓN: usar el costo variable
            if neighbor not in reached or new_cost < reached[neighbor].path_cost:
                new_actions = node.actions + [action]
                reached[neighbor] = Node(neighbor, parent=node, actions=new_actions, path_cost=new_cost)
                heapq.heappush(frontier, (new_cost + manhatan_distance(neighbor, end), reached[neighbor]))
    
    return None, None

# Laberinto más grande con diferentes tipos de terreno
large_maze = [
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", "S", ".", ".", "#", "W", "W", ".", ".", ".", ".", "#"],
    ["#", ".", "#", ".", "#", "W", "#", ".", "#", "#", ".", "#"],
    ["#", ".", "A", "A", "A", "W", ".", ".", ".", "M", ".", "#"],
    ["#", ".", "#", "#", ".", ".", "#", "#", ".", "M", ".", "#"],
    ["#", ".", ".", ".", ".", "A", "A", ".", ".", "M", ".", "#"],
    ["#", "#", "#", ".", "#", "#", "#", ".", "#", "M", ".", "#"],
    ["#", "W", "W", ".", ".", ".", ".", ".", ".", ".", ".", "#"],
    ["#", "W", "#", "#", "#", ".", "#", "#", "#", ".", "#", "#"],
    ["#", ".", ".", "M", ".", ".", ".", ".", ".", ".", "E", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#", "#", "#"]
]

path, actions = find_exit_with_costs(large_maze)
print("Camino encontrado:", path)
print("\nAcciones:", actions)
print(f"\nNúmero de pasos: {len(actions)}")

# Calcular el costo total del camino
terrain_costs = {".": 1, "W": 5, "A": 3, "M": 7, "S": 1, "E": 1}
total_cost = 0
for i in range(len(path)):
    terrain = large_maze[path[i][0]][path[i][1]]
    if i > 0:  # No contar el inicio
        total_cost += terrain_costs.get(terrain, 1)

print(f"Costo total del camino: {total_cost}")

**Conclusiones:**

**Limitación principal:** Costo uniforme para todos los espacios transitables, sin distinguir tipos de terreno.

**Modificaciones clave:**
1. Diccionario `terrain_costs` para mapear terreno a costo.
2. `get_neighbors` retorna `(neighbor, action_name, cost)`.
3. `new_cost = node.path_cost + cost` usa costo variable del terreno.

**Resultado:** El algoritmo ahora encuentra la ruta más económica, no solo la más corta en pasos.